In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributions as dists
import invertnn.pytorch.invertible_transforms as invertible
import invertnn.pytorch.orthogonal_transform as ortho

from torch.distributions.multivariate_normal import MultivariateNormal
import torch.distributions as dists 

from invertnn.pytorch.orthogonal_transform import OrthogonalTransform, DiagonalLinearTransform, InvertibleLinear, InvertibleLinear
from invertnn.pytorch.invertible_transforms import InvertibleModule, InvertibleConcatNoise, InvertibleCouplingLayer, InvertibleModuleTransform, InvertibleSequential, InvertibleBias, InvertibleView
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributions as dists
import invertnn.pytorch.invertible_transforms as invertible
import invertnn.pytorch.orthogonal_transform as ortho
import invertnn.pytorch.white_noise as white_noise
from unittest import TestCase
from collections import OrderedDict
import numpy as np
import torch
import math
import logging
from invertnn.pytorch.white_noise import AdaptiveInverseWhiteningTransformer

from torchvision.datasets import STL10, FashionMNIST
import math
import itertools
import pytorch_lightning as pl

In [2]:
torch.set_default_dtype(torch.float64)

In [3]:
fmnist = FashionMNIST(root='./FashionMNIST', download=True, train=True)
len(fmnist)

60000

In [4]:
fmnist[0][0]

In [5]:
#stl10 = STL10(root='./FashionMNIST', download=True, split='unlabeled')
#len(stl10)

In [6]:
#stl10[0][0]

In [7]:
def prod(iterable, startval=1):
    """Helper function, calculates the product of an iterable"""
    val = startval
    for i in iterable:
        val *= i
    return val


In [16]:

class SimpleInvertibleGenerator(nn.Module):
    
    def __init__(self, img_shape=(1,28,28)):
        super().__init__()
        n = prod(img_shape)
        self.imodel = invertible.InvertibleSequential(
            OrderedDict([
                    ('noise_1', InvertibleConcatNoise(dists.Normal(torch.zeros(10).to('cuda:0'), torch.ones(10).to('cuda:0')),
                                                      size=10, name='input_sample')),
                    ('linear_1', InvertibleLinear(n=10, bias=True)),
                    ('act_1', invertible.InvertibleSELU()),
                    ('linear_2', InvertibleLinear(n=10, bias=True)),
                    ('act_2', invertible.InvertibleSELU()),
                    ('linear_3', InvertibleLinear(n=10, bias=True)),
                    ('act_4', invertible.InvertibleSELU()),
                    ('linear_4', InvertibleLinear(n=10, bias=True)),
                    ('act_4', invertible.InvertibleSigmoid()),
                ]
            )
        )  
    
    def forward(self, inp):
        return self.imodel.forward(inp)
        
    

In [17]:
model = SimpleInvertibleGenerator()
model = model.to('cuda:0')

In [48]:

with invertible.InferenceContext() as ic:
    res = model.forward(torch.ones((2,0)).to('cuda:0'))
    
    input_sample = ic.side_inputs['input_sample']
    assert('input_sample' not in ic.reconstructed_side_inputs)
    restored = model.imodel.invert(res)
    
    restored_sample = ic.reconstructed_side_inputs['input_sample']
    error = (input_sample - restored_sample).abs().max()
    print("Max difference %f" % (error))

Calling noise_1 with input of shape torch.Size([2, 0])
Sampled shape torch.Size([2, 10])
Log Prob -24.939050
Calling linear_1 with input of shape torch.Size([2, 10])
Calling U with input of shape torch.Size([2, 10])
Calling S with input of shape torch.Size([2, 10])
Calling V with input of shape torch.Size([2, 10])
Calling act_1 with input of shape torch.Size([2, 10])
Calling linear_2 with input of shape torch.Size([2, 10])
Calling U with input of shape torch.Size([2, 10])
Calling S with input of shape torch.Size([2, 10])
Calling V with input of shape torch.Size([2, 10])
Calling act_2 with input of shape torch.Size([2, 10])
Calling linear_3 with input of shape torch.Size([2, 10])
Calling U with input of shape torch.Size([2, 10])
Calling S with input of shape torch.Size([2, 10])
Calling V with input of shape torch.Size([2, 10])
Calling act_4 with input of shape torch.Size([2, 10])
Calling linear_4 with input of shape torch.Size([2, 10])
Calling U with input of shape torch.Size([2, 10])
C

In [22]:
input_sample

tensor([[ 0.2056,  1.8611,  0.6372,  0.6467,  0.0790,  1.0157, -0.3161, -2.1013,
         -0.7796, -0.9845],
        [ 0.3448, -0.7178,  0.0283, -0.6479,  0.9502, -0.2250, -2.1621, -0.9078,
          0.6243, -1.8953]], device='cuda:0')

In [23]:
restored_sample

tensor([[ 0.2056,  1.8611,  0.6372,  0.6467,  0.0790,  1.0157, -0.3161, -2.1013,
         -0.7796, -0.9845],
        [ 0.3448, -0.7178,  0.0283, -0.6479,  0.9502, -0.2250, -2.1621, -0.9078,
          0.6243, -1.8953]], device='cuda:0', grad_fn=<SplitWithSizesBackward>)

In [16]:
%debug

> c:\users\vw6z1lw\pycharmprojects\invertnn\invertnn\pytorch\invertible_transforms.py(469)invert()
    467 
    468     def invert(self, output):
--> 469         sample, input = output.split(split_size_or_sections=[self.size, output.shape[1]-self.size], dim=1)
    470         restored_sample = sample
    471         restored_sample.log_prob = self.distribution.log_prob(self.restored_sample).sum()

ipdb> output.split
<bound method Tensor.split of tensor([[-0.4975, -0.7502, -1.6006, -2.1392,  0.5006,  0.1140, -0.4401,  0.0491,
         -0.1484, -0.9491],
        [ 0.0144,  0.3006,  0.2407, -0.0342, -0.2196,  0.2167, -0.2265, -2.0266,
         -0.9538,  0.9299]], device='cuda:0', grad_fn=<CopyBackwards>)>
ipdb> cont
